# Open IA

Primeiras aplicações utilizando a biblioteca `OpenIA`.

Objetivo:
* Utilizar a API da Open IA para fazer perguntas e gerar respostas por meio de um bot
* Documentar o processo de geração de texto

# Configurações iniciais

In [1]:
# Bibliotecas

import openai
from dotenv import load_dotenv
import os

In [3]:
# Acessando API

load_dotenv()
secret_key = os.getenv("OPEN_API_KEY")

In [5]:
# Instanciando 

client = openai.Client(api_key=secret_key)

## Prompt / message

**Parâmetro _role_**: define quem está enviando a mensagem

Há algumas opções. Pode conter: "user", "assistant", ou "system":

- `user` → Mensagem do usuário. Representa a entrada do usuário (você, a pessoa que está fazendo a pergunta).

- `assistant` → Resposta da IA (bot que responderá a pergunta) / gerada pelo modelo. OBS: Deve ser usada para armazenar as respostas da IA quando mantemos um histórico da conversa.


- `system` → Instruções para a IA. Define regras ou diretrizes para o modelo. Isso ajuda a configurar o comportamento da IA antes de ela responder ao usuário.É útil para definir estilo, tom de resposta ou contexto específico.


**Parâmetro _content_**: o conteúdo da mensagem enviada (texto enviado pelo usuário).

In [7]:
prompt = f"""
Defina um morango em 5 palavras
"""

In [8]:
message = [{"role": "user", "content":prompt}]

## Completion

_Completion_: Resposta gerada pelo modelo.

Parâmetros:
- `messages` → mensagem criada e definida anteriormente (seção acima)
- `model` → modelo do GPT (exemplos: gpt-3.5-turbo-0125, gpt-4, gpt-4-turbo)
- `max_tokens` → define o máximo de tokens (unidades de palavras/frases) que a resposta pode ter
- `temperature` → controla o grau de aleatoriedade da resposta (varia de 0 a 1). 0: respostas deterministicas e precisas. 1: respostas criativas e variadas.

In [9]:
completion = client.chat.completions.create(
    messages=message,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

A variável vai retornar a resposta do modelo. O que temos de interesse está em `message=ChatCompletionMessage(content=)`.

In [10]:
completion

ChatCompletion(id='chatcmpl-BHGp22Sx46nzqlp1s0yL1ZxnY1k0Z', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Fruta vermelha, doce e suculenta.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1743455940, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=19, total_tokens=33, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [11]:
# Isolando a parte de interesse

completion.choices[0].message.content

'Fruta vermelha, doce e suculenta.'

- Adicionando mais coisas na lista de perguntas
- Trocar agora o role para o assistant (que no nosso caso, é o bot que responde a pergunta)
- Sempre que o role for 'assistant' = bot
- Sempre que o role for 'user' = pergunta de entrada do usuário

In [12]:
# Armazenando resposta do bot

message.append({'role':'assistant','content':completion.choices[0].message.content})

In [13]:
# Inserindo nova pergunta

message.append({'role':'user','content':'e tem mais alguma cor nessa fruta?'})

In [14]:
# Resposta gerada pelo modelo

completion = client.chat.completions.create(
    messages=message,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

In [15]:
# Isolando resposta do modelo

completion.choices[0].message.content

'Sim, o morango também pode ter tons de rosa e até mesmo branco.'

In [16]:
# Armazenando a resposta do bot

message.append({'role':'assistant', 'content':completion.choices[0].message.content})

In [17]:
# Exibindo o histórico

message

[{'role': 'user', 'content': '\nDefina um morango em 5 palavras\n'},
 {'role': 'assistant', 'content': 'Fruta vermelha, doce e suculenta.'},
 {'role': 'user', 'content': 'e tem mais alguma cor nessa fruta?'},
 {'role': 'assistant',
  'content': 'Sim, o morango também pode ter tons de rosa e até mesmo branco.'}]

## Automação do script

In [18]:
# Criando uma função com as utilizações acima

def geracao_texto(mensagens,model='gpt-3.5-turbo-0125',max_tokens=1000,temperature=0):
    resposta = client.chat.completions.create(
        messages=mensagens,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )
    mensagens.append({'role':'assistant','content': resposta.choices[0].message.content})
    return mensagens

In [19]:
msg1 = [{'role':'user','content':'Defina o que é API em 5 palavras'}]
teste = geracao_texto(msg1)

In [20]:
teste

[{'role': 'user', 'content': 'Defina o que é API em 5 palavras'},
 {'role': 'assistant', 'content': 'Interface de programação de aplicativos.'}]

### Mais parâmetros interessantes

`Store` e `Stream`

`Store`: parâmetro que controla se a resposta gerada pela API deve ser armazenada (ou não) no sistema. Isso pode ser útil, por exemplo, para permitir que você armazene as respostas para análises posteriores ou para o histórico de um chat.

* Usar store=True para armazenar a resposta gerada, algo que pode ser útil em situações onde se  quer manter o histórico de interações
* Na API da OpenAI, o valor padrão para o parâmetro store é True. Isso significa que, por padrão, as respostas geradas pela API serão armazenadas automaticamente.
* Se não precisar armazenar a resposta para análise futura, pode-se definir store=False.

`Stream`: parâmetro que permite que você receba a resposta de forma parcial e contínua enquanto a geração ainda está acontecendo. Em vez de esperar até que a resposta completa seja gerada, com o streaming, a resposta começa a ser enviada imediatamente, à medida que o modelo gera cada parte do texto.

* Usar o stream=True se quiser visualizar a resposta sendo gerada em tempo real, por exemplo, em um chatbot ao vivo ou quando você precisa exibir a resposta enquanto ela ainda está sendo processada
* Isso cria um efeito de "digitação" do modelo, onde o texto aparece dinamicamente enquanto é gerado.

### Utilizando Stream

In [21]:
message = [{'role':'user','content':'Diga quais as cores da bandeira do Brasil e o que cada uma representa'}]
completion = client.chat.completions.create(
    messages=message,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0,
    stream=True # Testando o stream aqui
)

In [22]:
completion # gera um objeto stream

In [23]:
# OBS: só funciona com for

for completion_stream in completion:
    text = completion_stream.choices[0].delta.content
    if text:
        print(text, end="")

A bandeira do Brasil é composta por quatro cores: verde, amarelo, azul e branco.

- Verde: representa as matas e florestas do país, simbolizando a esperança e a riqueza natural.
- Amarelo: simboliza as riquezas minerais do Brasil, como o ouro, e representa a riqueza do país.
- Azul: representa o céu e os rios do Brasil, simbolizando a união entre o céu e a água.
- Branco: simboliza a paz e a pureza, representando a harmonia entre os povos do Brasil.